In [1]:
# =============================================================================
# COLETA DE DADOS COMPLETA - TODOS OS PAÍSES DA BASE DAI
# Projeto: Escolaridade como Proteção contra Desemprego Tecnológico
# VERSÃO 4 - DATASET COMPLETO
# =============================================================================

import pandas as pd
import requests
import numpy as np
import warnings
import io
warnings.filterwarnings('ignore')

print("="*80)
print("COLETA DE DADOS COMPLETA - TODOS OS PAÍSES COM DAI DISPONÍVEL")
print("World Bank - Digital Adoption Index + Human Capital Index")
print("="*80)



COLETA DE DADOS COMPLETA - TODOS OS PAÍSES COM DAI DISPONÍVEL
World Bank - Digital Adoption Index + Human Capital Index


In [2]:
# =============================================================================
# 1. CONFIGURAÇÃO DAS FONTES DE DADOS
# =============================================================================

FONTES_DADOS = {
    'world_bank_api': 'https://api.worldbank.org/v2',
    'dai_excel_url': 'https://thedocs.worldbank.org/en/doc/625521534508595697-0050022018/original/DAIforweb.xlsx',
    'hci_databank': 'https://databank.worldbank.org/source/human-capital-index',
    'wdi_base': 'https://data.worldbank.org/indicator'
}

# Indicadores World Bank
INDICADORES_WB = {
    # Econômicos
    'gdp_per_capita': 'NY.GDP.PCAP.CD',
    'population': 'SP.POP.TOTL',
    'gini_index': 'SI.POV.GINI',
    
    # Human Capital Index
    'hci_overall': 'HD.HCI.OVRL',
    'hci_expected_years': 'HD.HCI.EYRS',
    'hci_learning': 'HD.HCI.LAYS',
    
    # Digitais/Tecnológicos
    'internet_users': 'IT.NET.USER.ZS',
    'mobile_subscriptions': 'IT.CEL.SETS.P2',
    
    # Educação
    'tertiary_enrollment': 'SE.TER.ENRR',
    'research_development': 'GB.XPD.RSDV.GD.ZS',
    
    # Governança
    'government_effectiveness': 'GE.EST',
    'regulatory_quality': 'RQ.EST'
}

print(f"📊 Configuração:")
print(f"   • Modo: COLETA COMPLETA (todos os países com DAI)")
print(f"   • {len(INDICADORES_WB)} indicadores World Bank")



📊 Configuração:
   • Modo: COLETA COMPLETA (todos os países com DAI)
   • 12 indicadores World Bank


In [3]:
# =============================================================================
# 2. COLETA DAI - TODOS OS PAÍSES DO EXCEL
# =============================================================================

print(f"\n{'='*60}")
print("FASE 1: COLETANDO DAI (DIGITAL ADOPTION INDEX)")
print(f"{'='*60}")

print("📥 Baixando arquivo Excel do World Bank...")
print(f"   URL: {FONTES_DADOS['dai_excel_url']}")

try:
    response = requests.get(FONTES_DADOS['dai_excel_url'], timeout=60)
    response.raise_for_status()
    
    excel_file = io.BytesIO(response.content)
    df_dai_raw = pd.read_excel(excel_file, sheet_name=0)
    
    print(f"   ✅ Arquivo carregado! Shape: {df_dai_raw.shape}")
    print(f"   📋 Colunas: {list(df_dai_raw.columns)}")
    
    # Mapeamento expandido: Nome País → Código ISO (todos os principais)
    # Usando biblioteca pycountry ou mapeamento manual expandido
    from difflib import get_close_matches
    
    # Dicionário expandido com mais de 200 países
    COUNTRY_NAME_TO_ISO = {
        # Principais economias e variações
        'United States': 'USA', 'United States of America': 'USA',
        'United Kingdom': 'GBR', 'UK': 'GBR', 'Britain': 'GBR',
        'China': 'CHN', "China, People's Republic of": 'CHN',
        'Germany': 'DEU', 'France': 'FRA', 'Italy': 'ITA', 'Spain': 'ESP',
        'Japan': 'JPN', 'Canada': 'CAN', 'Australia': 'AUS',
        'Korea, Rep.': 'KOR', 'South Korea': 'KOR', 'Korea': 'KOR',
        'Brazil': 'BRA', 'India': 'IND', 'Mexico': 'MEX',
        'Russia': 'RUS', 'Russian Federation': 'RUS',
        'Indonesia': 'IDN', 'Netherlands': 'NLD', 'Saudi Arabia': 'SAU',
        'Turkey': 'TUR', 'Türkiye': 'TUR', 'Switzerland': 'CHE',
        'Poland': 'POL', 'Belgium': 'BEL', 'Sweden': 'SWE',
        'Argentina': 'ARG', 'Norway': 'NOR', 'Austria': 'AUT',
        'Iran': 'IRN', 'Iran, Islamic Rep.': 'IRN',
        'Thailand': 'THA', 'Israel': 'ISR', 'Singapore': 'SGP',
        'Philippines': 'PHL', 'Malaysia': 'MYS', 'South Africa': 'ZAF',
        'Colombia': 'COL', 'Pakistan': 'PAK', 'Chile': 'CHL',
        'Finland': 'FIN', 'Bangladesh': 'BGD', 'Egypt': 'EGY',
        'Egypt, Arab Rep.': 'EGY', 'Vietnam': 'VNM', 'Viet Nam': 'VNM',
        'Czech Republic': 'CZE', 'Czechia': 'CZE', 'Portugal': 'PRT',
        'Romania': 'ROU', 'Peru': 'PER', 'New Zealand': 'NZL',
        'Greece': 'GRC', 'Iraq': 'IRQ', 'Algeria': 'DZA',
        'Qatar': 'QAT', 'Kazakhstan': 'KAZ', 'Hungary': 'HUN',
        'Kuwait': 'KWT', 'Morocco': 'MAR', 'Slovakia': 'SVK',
        'Ecuador': 'ECU', 'Puerto Rico': 'PRI', 'Kenya': 'KEN',
        'Ethiopia': 'ETH', 'Dominican Republic': 'DOM', 'Guatemala': 'GTM',
        'Oman': 'OMN', 'Venezuela': 'VEN', 'Venezuela, RB': 'VEN',
        'Luxembourg': 'LUX', 'Panama': 'PAN', 'Ghana': 'GHA',
        'Croatia': 'HRV', 'Costa Rica': 'CRI', 'Uruguay': 'URY',
        'Belarus': 'BLR', 'Bulgaria': 'BGR', 'Serbia': 'SRB',
        'Azerbaijan': 'AZE', 'Sri Lanka': 'LKA', 'Lithuania': 'LTU',
        'Slovenia': 'SVN', 'Uzbekistan': 'UZB', 'Tunisia': 'TUN',
        'Libya': 'LBY', 'Jordan': 'JOR', 'Bolivia': 'BOL',
        'Cameroon': 'CMR', 'Paraguay': 'PRY', 'Latvia': 'LVA',
        'Uganda': 'UGA', 'Estonia': 'EST', 'El Salvador': 'SLV',
        'Nepal': 'NPL', 'Honduras': 'HND', 'Cambodia': 'KHM',
        'Cyprus': 'CYP', 'Senegal': 'SEN', 'Bosnia and Herzegovina': 'BIH',
        'Papua New Guinea': 'PNG', 'Jamaica': 'JAM', 'Albania': 'ALB',
        'Gabon': 'GAB', 'Botswana': 'BWA', 'Georgia': 'GEO',
        'Mali': 'MLI', 'Malta': 'MLT', 'Mongolia': 'MNG',
        'Armenia': 'ARM', 'Mozambique': 'MOZ', 'Burkina Faso': 'BFA',
        'Mauritius': 'MUS', 'Namibia': 'NAM', 'Madagascar': 'MDG',
        'North Macedonia': 'MKD', 'Macedonia': 'MKD', 'Iceland': 'ISL',
        'Nicaragua': 'NIC', 'Chad': 'TCD', 'Zimbabwe': 'ZWE',
        'Moldova': 'MDA', 'Malawi': 'MWI', 'Afghanistan': 'AFG',
        'Rwanda': 'RWA', 'Kyrgyzstan': 'KGZ', 'Niger': 'NER',
        'Tajikistan': 'TJK', 'Haiti': 'HTI', 'Guinea': 'GIN',
        'Benin': 'BEN', 'Burundi': 'BDI', 'Denmark': 'DNK',
        'Lebanon': 'LBN', 'Ukraine': 'UKR', 'Myanmar': 'MMR',
        'Tanzania': 'TZA', 'Yemen': 'YEM', 'Yemen, Rep.': 'YEM',
        'Sudan': 'SDN', 'Angola': 'AGO', 'Syria': 'SYR',
        'Syrian Arab Republic': 'SYR', 'Zambia': 'ZMB',
        'Congo, Dem. Rep.': 'COD', 'Congo, Rep.': 'COG',
        'Laos': 'LAO', "Lao PDR": 'LAO', 'Sierra Leone': 'SLE',
        'Togo': 'TGO', 'Liberia': 'LBR', 'Mauritania': 'MRT',
        'Central African Republic': 'CAF', 'Eritrea': 'ERI',
        'Gambia': 'GMB', 'The Gambia': 'GMB'
    }
    
    # Função de mapeamento inteligente, retorna codigo iso do país
    def map_country_to_iso(country_name):
        if pd.isna(country_name):
            return None
        
        country_str = str(country_name).strip()
        
        # Match direto
        if country_str in COUNTRY_NAME_TO_ISO:
            return COUNTRY_NAME_TO_ISO[country_str]
        
        # Match case-insensitive
        for name, code in COUNTRY_NAME_TO_ISO.items():
            if name.lower() == country_str.lower():
                return code
        
        # Match parcial
        for name, code in COUNTRY_NAME_TO_ISO.items():
            if name.lower() in country_str.lower() or country_str.lower() in name.lower():
                return code
        
        return None
    
    # Filtrar dados de 2016
    col_country = 'country'
    col_year = 'Year'
    
    if col_year in df_dai_raw.columns:
        df_dai_2016 = df_dai_raw[df_dai_raw[col_year] == 2016].copy()
        print(f"   📅 Filtrados dados de 2016: {len(df_dai_2016)} países")
    else:
        df_dai_2016 = df_dai_raw.copy()
    
    # Mapear países
    df_dai_2016['country_code'] = df_dai_2016[col_country].apply(map_country_to_iso)
    
    # Remover países sem mapeamento
    paises_nao_mapeados = df_dai_2016[df_dai_2016['country_code'].isna()][col_country].unique()
    if len(paises_nao_mapeados) > 0:
        print(f"   ⚠️  {len(paises_nao_mapeados)} países não mapeados (serão ignorados)")
        if len(paises_nao_mapeados) <= 10:
            print(f"      Não mapeados: {list(paises_nao_mapeados)}")
    
    df_dai_2016 = df_dai_2016[df_dai_2016['country_code'].notna()].copy()
    
    # Criar DataFrame processado
    df_dai_final = pd.DataFrame({
        'country_code': df_dai_2016['country_code'],
        'country_name': df_dai_2016[col_country],
        'dai_overall_2016': df_dai_2016['Digital Adoption Index']
    })
    
    # Adicionar dimensões
    if 'DAI People Sub-index' in df_dai_2016.columns:
        df_dai_final['dai_people_2016'] = df_dai_2016['DAI People Sub-index'].values
    
    if 'DAI Government Sub-index' in df_dai_2016.columns:
        df_dai_final['dai_government_2016'] = df_dai_2016['DAI Government Sub-index'].values
    
    if 'DAI Business Sub-index' in df_dai_2016.columns:
        df_dai_final['dai_business_2016'] = df_dai_2016['DAI Business Sub-index'].values
    
    # Remover duplicatas (manter primeira ocorrência)
    df_dai_final = df_dai_final.drop_duplicates(subset=['country_code'], keep='first')
    
    # Arredondar
    for col in df_dai_final.columns:
        if col not in ['country_code', 'country_name']:
            df_dai_final[col] = df_dai_final[col].round(3)
    
    print(f"   ✅ DAI coletado: {len(df_dai_final)} países")
    print(f"   📊 Variáveis: {list(df_dai_final.columns)}")
    
    # Lista de países para coleta da API
    PAISES_DAI = df_dai_final['country_code'].unique().tolist()
    print(f"   🌍 Países para coleta API: {len(PAISES_DAI)}")
    
except Exception as e:
    print(f"   ❌ Erro ao processar DAI: {e}")
    print(f"   🛑 Encerrando execução")
    raise




FASE 1: COLETANDO DAI (DIGITAL ADOPTION INDEX)
📥 Baixando arquivo Excel do World Bank...
   URL: https://thedocs.worldbank.org/en/doc/625521534508595697-0050022018/original/DAIforweb.xlsx
   ✅ Arquivo carregado! Shape: (366, 6)
   📋 Colunas: ['country', 'Year', 'Digital Adoption Index', 'DAI Business Sub-index', 'DAI People Sub-index', 'DAI Government Sub-index']
   📅 Filtrados dados de 2016: 183 países
   ⚠️  39 países não mapeados (serão ignorados)
   ✅ DAI coletado: 138 países
   📊 Variáveis: ['country_code', 'country_name', 'dai_overall_2016', 'dai_people_2016', 'dai_government_2016', 'dai_business_2016']
   🌍 Países para coleta API: 138


In [4]:
# =============================================================================
# 3. COLETA DE DADOS VIA API WORLD BANK
# =============================================================================

print(f"\n{'='*60}")
print("FASE 2: COLETANDO DADOS VIA API WORLD BANK")
print(f"{'='*60}")

def coletar_dados_worldbank_batch(paises, indicador, anos='2018:2020', batch_size=50):
    """
    Coleta dados em batches para evitar URLs muito longas
    """
    all_data = []
    
    # Dividir países em batches
    for i in range(0, len(paises), batch_size):
        batch = paises[i:i+batch_size]
        paises_str = ';'.join(batch)
        
        url = f"{FONTES_DADOS['world_bank_api']}/country/{paises_str}/indicator/{indicador}"
        params = {'date': anos, 'format': 'json', 'per_page': 1000}
        
        try:
            response = requests.get(url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            if len(data) >= 2 and data[1]:
                for item in data[1]:
                    if item['value'] is not None:
                        all_data.append({
                            'country_code': item['countryiso3code'],
                            'country_name': item['country']['value'],
                            'year': int(item['date']),
                            'value': float(item['value'])
                        })
        except Exception as e:
            print(f"      ⚠️  Erro no batch {i//batch_size + 1}: {e}")
            continue
    
    if all_data:
        df = pd.DataFrame(all_data)
        # Pegar valor mais recente
        df_latest = df.loc[df.groupby('country_code')['year'].idxmax()]
        return df_latest[['country_code', 'country_name', 'year', 'value']]
    
    return pd.DataFrame()

# Coletar indicadores
dados_coletados = {'dai': df_dai_final}

for nome_var, codigo in INDICADORES_WB.items():
    print(f"🔍 Coletando: {nome_var} ({codigo})")
    df_indicador = coletar_dados_worldbank_batch(PAISES_DAI, codigo)
    
    if not df_indicador.empty:
        dados_coletados[nome_var] = df_indicador
        print(f"   ✅ {len(df_indicador)} países coletados")
    else:
        print(f"   ⚠️  Nenhum dado disponível")

print(f"\n✅ Coleta API concluída: {len(dados_coletados)} indicadores")




FASE 2: COLETANDO DADOS VIA API WORLD BANK
🔍 Coletando: gdp_per_capita (NY.GDP.PCAP.CD)
   ✅ 137 países coletados
🔍 Coletando: population (SP.POP.TOTL)
   ✅ 138 países coletados
🔍 Coletando: gini_index (SI.POV.GINI)
   ✅ 94 países coletados
🔍 Coletando: hci_overall (HD.HCI.OVRL)
   ✅ 135 países coletados
🔍 Coletando: hci_expected_years (HD.HCI.EYRS)
   ✅ 135 países coletados
🔍 Coletando: hci_learning (HD.HCI.LAYS)
   ✅ 135 países coletados
🔍 Coletando: internet_users (IT.NET.USER.ZS)
   ✅ 137 países coletados
🔍 Coletando: mobile_subscriptions (IT.CEL.SETS.P2)
   ✅ 138 países coletados
🔍 Coletando: tertiary_enrollment (SE.TER.ENRR)
   ✅ 118 países coletados
🔍 Coletando: research_development (GB.XPD.RSDV.GD.ZS)
   ✅ 91 países coletados
🔍 Coletando: government_effectiveness (GE.EST)
   ✅ 138 países coletados
🔍 Coletando: regulatory_quality (RQ.EST)
   ✅ 138 países coletados

✅ Coleta API concluída: 13 indicadores


In [5]:
# =============================================================================
# 4. CONSOLIDAÇÃO DO DATASET
# =============================================================================

print(f"\n{'='*60}")
print("FASE 3: CONSOLIDAÇÃO DO DATASET FINAL")
print(f"{'='*60}")

# Base: países com DAI
df_final = df_dai_final[['country_code', 'country_name', 'dai_overall_2016', 
                          'dai_people_2016', 'dai_government_2016', 'dai_business_2016']].copy()

# Merge com outros indicadores
primeiro_merge = True
for nome_var, df_dados in dados_coletados.items():
    if nome_var == 'dai':
        continue
    
    if not df_dados.empty and 'value' in df_dados.columns:
        print(f"   🔗 Merging: {nome_var}")
        
        if primeiro_merge and 'country_name' not in df_final.columns:
            df_temp = df_dados[['country_code', 'country_name', 'value']].rename(columns={'value': nome_var})
            df_final = df_final.merge(df_temp, on='country_code', how='left')
            primeiro_merge = False
        else:
            df_temp = df_dados[['country_code', 'value']].rename(columns={'value': nome_var})
            df_final = df_final.merge(df_temp, on='country_code', how='left')

# Adicionar classificações do World Bank
print("   📊 Adicionando classificações World Bank...")

# Mapear grupos de renda baseado em PIB per capita (critérios WB 2020)
def classificar_renda(gdp):
    if pd.isna(gdp):
        return 'Unknown'
    elif gdp >= 12536:
        return 'HIC'  # High income
    elif gdp >= 4046:
        return 'UMC'  # Upper middle
    elif gdp >= 1036:
        return 'LMC'  # Lower middle
    else:
        return 'LIC'  # Low income

if 'gdp_per_capita' in df_final.columns:
    df_final['income_group'] = df_final['gdp_per_capita'].apply(classificar_renda)

# Adicionar regiões (simplificado - pode ser expandido com API adicional)
df_final['region'] = 'Unknown'  # Placeholder - pode usar API adicional

# Metadados
df_final['data_coleta'] = pd.Timestamp.now().strftime('%Y-%m-%d')
df_final['versao'] = 'v4_completo'

print(f"\n✅ Dataset final consolidado!")
print(f"   📏 Dimensões: {df_final.shape[0]} países × {df_final.shape[1]} variáveis")




FASE 3: CONSOLIDAÇÃO DO DATASET FINAL
   🔗 Merging: gdp_per_capita
   🔗 Merging: population
   🔗 Merging: gini_index
   🔗 Merging: hci_overall
   🔗 Merging: hci_expected_years
   🔗 Merging: hci_learning
   🔗 Merging: internet_users
   🔗 Merging: mobile_subscriptions
   🔗 Merging: tertiary_enrollment
   🔗 Merging: research_development
   🔗 Merging: government_effectiveness
   🔗 Merging: regulatory_quality
   📊 Adicionando classificações World Bank...

✅ Dataset final consolidado!
   📏 Dimensões: 138 países × 22 variáveis


In [6]:
# =============================================================================
# 5. ANÁLISE DE QUALIDADE
# =============================================================================

print(f"\n{'='*60}")
print("ANÁLISE DE QUALIDADE DOS DADOS")
print(f"{'='*60}")

print(f"\n📊 COMPLETUDE POR VARIÁVEL:")
for col in df_final.columns:
    if col not in ['country_code', 'country_name', 'region', 'data_coleta', 'versao']:
        total = len(df_final)
        completos = df_final[col].notna().sum()
        pct = (completos / total) * 100
        status = "✅" if pct >= 80 else "⚠️" if pct >= 50 else "❌"
        print(f"   {status} {col}: {completos}/{total} ({pct:.1f}%)")

# Filtrar países com dados essenciais
print(f"\n🔍 FILTRAGEM DE QUALIDADE:")
df_completo = df_final[
    df_final['dai_overall_2016'].notna() & 
    df_final['hci_overall'].notna()
].copy()

print(f"   Países com DAI e HCI: {len(df_completo)}")
print(f"   Países removidos: {len(df_final) - len(df_completo)}")




ANÁLISE DE QUALIDADE DOS DADOS

📊 COMPLETUDE POR VARIÁVEL:
   ✅ dai_overall_2016: 138/138 (100.0%)
   ✅ dai_people_2016: 138/138 (100.0%)
   ✅ dai_government_2016: 138/138 (100.0%)
   ✅ dai_business_2016: 138/138 (100.0%)
   ✅ gdp_per_capita: 137/138 (99.3%)
   ✅ population: 138/138 (100.0%)
   ⚠️ gini_index: 94/138 (68.1%)
   ✅ hci_overall: 135/138 (97.8%)
   ✅ hci_expected_years: 135/138 (97.8%)
   ✅ hci_learning: 135/138 (97.8%)
   ✅ internet_users: 137/138 (99.3%)
   ✅ mobile_subscriptions: 138/138 (100.0%)
   ✅ tertiary_enrollment: 118/138 (85.5%)
   ⚠️ research_development: 91/138 (65.9%)
   ✅ government_effectiveness: 138/138 (100.0%)
   ✅ regulatory_quality: 138/138 (100.0%)
   ✅ income_group: 138/138 (100.0%)

🔍 FILTRAGEM DE QUALIDADE:
   Países com DAI e HCI: 135
   Países removidos: 3


In [8]:
# =============================================================================
# 6. EXPORTAÇÃO
# =============================================================================

print(f"\n{'='*60}")
print("EXPORTAÇÃO DOS DATASETS")
print(f"{'='*60}")

# Dataset completo (todos os países com DAI)
nome_completo = f"dataset_completo_dai_{pd.Timestamp.now().strftime('%Y%m%d')}.csv"
df_final.to_csv(nome_completo, index=False, encoding='utf-8')
print(f"   💾 Dataset completo: {nome_completo}")
print(f"      ({len(df_final)} países, {len(df_final.columns)} variáveis)")

# Dataset filtrado (apenas países com DAI e HCI)
nome_filtrado = f"dataset_analise_dai_hci_{pd.Timestamp.now().strftime('%Y%m%d')}.csv"
df_completo.to_csv(nome_filtrado, index=False, encoding='utf-8')
print(f"   💾 Dataset para análise: {nome_filtrado}")
print(f"      ({len(df_completo)} países, {len(df_completo.columns)} variáveis)")

# Excel
nome_excel = nome_filtrado.replace('.csv', '.xlsx')
df_completo.to_excel(nome_excel, index=False, engine='openpyxl')
print(f"   📊 Dataset Excel: {nome_excel}")

print(f"\n{'='*80}")
print(f"✅ COLETA COMPLETA FINALIZADA COM SUCESSO!")
print(f"{'='*80}")
print(f"\n📊 RESUMO FINAL:")
print(f"   • Países com DAI: {len(df_final)}")
print(f"   • Países com DAI + HCI: {len(df_completo)}")
print(f"   • Variáveis coletadas: {len(df_final.columns)}")
print(f"   • Arquivos gerados: 3 (2 CSV + 1 Excel)")
print(f"\n🎯 Dataset pronto para análises avançadas!")

# Retornar dataset para uso
df_completo


EXPORTAÇÃO DOS DATASETS


   💾 Dataset completo: dataset_completo_dai_20251003.csv
      (138 países, 22 variáveis)
   💾 Dataset para análise: dataset_analise_dai_hci_20251003.csv
      (135 países, 22 variáveis)
   📊 Dataset Excel: dataset_analise_dai_hci_20251003.xlsx

✅ COLETA COMPLETA FINALIZADA COM SUCESSO!

📊 RESUMO FINAL:
   • Países com DAI: 138
   • Países com DAI + HCI: 135
   • Variáveis coletadas: 22
   • Arquivos gerados: 3 (2 CSV + 1 Excel)

🎯 Dataset pronto para análises avançadas!


,country_code,country_name,dai_overall_2016,dai_people_2016,dai_government_2016,dai_business_2016,gdp_per_capita,population,gini_index,hci_overall,...,internet_users,mobile_subscriptions,tertiary_enrollment,research_development,government_effectiveness,regulatory_quality,income_group,region,data_coleta,versao
0,AFG,Afghanistan,0.343,0.123,0.564,0.342,510.787063,39068979.0,NaN,0.400284,...,17.0,58.0461,10.858440,NaN,-1.611539,-1.389163,LIC,Unknown,2025-10-03,v4_completo
1,ALB,Albania,0.608,0.464,0.739,0.621,5370.777500,2837849.0,29.4,0.634251,...,72.2,91.1881,61.585300,NaN,-0.155121,0.221967,UMC,Unknown,2025-10-03,v4_completo
2,DZA,Algeria,0.431,0.420,0.375,0.498,3743.541952,44042091.0,NaN,0.534556,...,63.5,103.4370,53.614029,NaN,-0.573643,-1.355300,LMC,Unknown,2025-10-03,v4_completo
3,AGO,Angola,0.335,0.132,0.464,0.409,1449.922867,33451132.0,51.3,0.362405,...,36.6,43.7804,11.061359,NaN,-1.257485,-0.856853,LMC,Unknown,2025-10-03,v4_completo
4,ARG,Argentina,0.686,0.634,0.733,0.690,8535.599380,45191965.0,42.7,0.602145,...,85.5,121.1810,99.266228,0.54126,-0.254262,-0.498344,UMC,Unknown,2025-10-03,v4_completo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,UZB,Uzbekistan,0.401,0.313,0.531,0.359,1978.280519,33586372.0,NaN,0.622806,...,71.1,99.4064,15.972095,0.13850,-0.615506,-0.999216,LMC,Unknown,2025-10-03,v4_completo
134,VNM,Vietnam,0.521,0.431,0.539,0.594,3534.039535,98079191.0,36.8,0.689965,...,70.3,141.6560,28.506929,0.41652,0.193558,-0.236916,LMC,Unknown,2025-10-03,v4_completo
135,YEM,"Yemen, Rep.",0.255,0.157,0.356,0.253,559.564676,36134863.0,NaN,0.372784,...,13.8,42.0038,NaN,NaN,-2.362375,-1.857522,LIC,Unknown,2025-10-03,v4_completo
136,ZMB,Zambia,0.344,0.184,0.519,0.330,951.644317,19059395.0,NaN,0.396928,...,24.5,100.2350,NaN,NaN,-0.824682,-0.687794,LIC,Unknown,2025-10-03,v4_completo


In [9]:
df_completo.to_excel('dados.xlsx')